In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

In [40]:
train_data = pd.read_csv('train.csv')

In [3]:
train_data.shape

(1458644, 11)

In [4]:
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [5]:
def findDistance(cols):
    x1 = cols[0]
    y1 = cols[1]
    x2 = cols[2]
    y2 = cols[3]
    distance = ((y2-y1)**2 + (x2-x1)**2)**1/2
    return distance

In [6]:
distance = train_data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].apply(findDistance,axis=1)

In [7]:
train_data['distance'] = distance

In [8]:
train_data.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1, inplace=True)

In [9]:
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,N,455,0.000156
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,N,663,0.000209
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,N,2124,0.001796
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,N,429,0.000090
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,N,435,0.000057


In [10]:
vend_id = pd.get_dummies(train_data['vendor_id'],drop_first=True)

In [11]:
flag = pd.get_dummies(train_data['store_and_fwd_flag'],drop_first=True)

In [12]:
train_data.drop(['vendor_id','store_and_fwd_flag'],axis=1, inplace=True)

In [13]:
train_data = pd.concat([train_data,vend_id,flag],axis=1)

In [14]:
train_data.head()

,id,pickup_datetime,dropoff_datetime,passenger_count,trip_duration,distance,2,Y
0,id2875421,2016-03-14 17:24:55,2016-03-14 17:32:30,1,455,0.000156,1,0
1,id2377394,2016-06-12 00:43:35,2016-06-12 00:54:38,1,663,0.000209,0,0
2,id3858529,2016-01-19 11:35:24,2016-01-19 12:10:48,1,2124,0.001796,1,0
3,id3504673,2016-04-06 19:32:31,2016-04-06 19:39:40,1,429,0.000090,1,0
4,id2181028,2016-03-26 13:30:55,2016-03-26 13:38:10,1,435,0.000057,1,0


In [15]:
train_data.drop('dropoff_datetime',axis = 1,inplace=True)

In [16]:
def find_weekday(cols):
    d = datetime.datetime.strptime(cols[0], '%Y-%m-%d %H:%M:%S')
    weekday = d.strftime("%w")
    return weekday

In [17]:
weekday = train_data[['pickup_datetime']].apply(find_weekday,axis=1)

In [18]:
def find_tod(cols):
    d = datetime.datetime.strptime(cols[0], '%Y-%m-%d %H:%M:%S')
    hour = int(d.strftime("%H"))
    tod = ""
    if hour>=0 and hour<6:
        tod = "em"
    elif hour>=6 and hour<12:
        tod = "m"
    elif hour>=12 and hour<18:
        tod = "af"
    else:
        tod = "n"
    return tod

In [19]:
tod = train_data[['pickup_datetime']].apply(find_tod,axis=1)

In [20]:
train_data.drop('pickup_datetime',axis=1, inplace=True)

In [21]:
weekday = pd.get_dummies(weekday)

In [22]:
tod = pd.get_dummies(tod)

In [23]:
weekday.drop(['0'],axis=1,inplace=True)

In [24]:
train_data = pd.concat([train_data,weekday,tod],axis=1)

In [25]:
train_data.drop('id',axis=1,inplace=True)

In [26]:
train_data.head()

,passenger_count,trip_duration,distance,2,Y,1,2,3,4,5,6,af,em,m,n
0,1,455,0.000156,1,0,1,0,0,0,0,0,1,0,0,0
1,1,663,0.000209,0,0,0,0,0,0,0,0,0,1,0,0
2,1,2124,0.001796,1,0,0,1,0,0,0,0,0,0,1,0
3,1,429,0.000090,1,0,0,0,1,0,0,0,0,0,0,1
4,1,435,0.000057,1,0,0,0,0,0,0,1,1,0,0,0


In [27]:
X = train_data.drop('trip_duration',axis=1)
y = train_data['trip_duration']

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
scaler = StandardScaler()

In [30]:
scaler.fit(X)
X = scaler.transform(X)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

In [33]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
rfs = RandomForestRegressor(n_estimators=10)

In [35]:
rfs.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [37]:
train_data = 0

In [41]:
predictions = rfs.predict(X_test)

In [38]:
import gc

In [39]:
gc.collect()

7

In [44]:
pre = pd.Series(predictions)

In [53]:
type(y_test)

pandas.core.series.Series

In [75]:
res = y_test.values - pre

In [95]:
(sum((np.log(pre+1) - np.log(y_test.values + 1))**2)/437594)**1/2

0.19385819088944692

Cleaning test data

In [113]:
test_data = pd.read_csv('test.csv')

In [114]:
test_data.shape

(625134, 9)

In [115]:
test_data.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [116]:
distance = test_data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].apply(findDistance,axis=1)

In [117]:
test_data['distance'] = distance

In [118]:
test_data.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1, inplace=True)

In [119]:
test_data.head()

,id,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,distance
0,id3004672,1,2016-06-30 23:59:58,1,N,0.000306
1,id3505355,1,2016-06-30 23:59:53,1,N,0.000312
2,id1217141,1,2016-06-30 23:59:47,1,N,0.000096
3,id2150126,2,2016-06-30 23:59:41,1,N,0.001319
4,id1598245,1,2016-06-30 23:59:33,1,N,0.000053


In [121]:
vend_id = pd.get_dummies(test_data['vendor_id'],drop_first=True)

In [122]:
flag = pd.get_dummies(test_data['store_and_fwd_flag'],drop_first=True)

In [123]:
test_data.drop(['vendor_id','store_and_fwd_flag'],axis=1, inplace=True)

In [124]:
test_data = pd.concat([test_data,vend_id,flag],axis=1)

In [125]:
test_data.head()

,id,pickup_datetime,passenger_count,distance,2,Y
0,id3004672,2016-06-30 23:59:58,1,0.000306,0,0
1,id3505355,2016-06-30 23:59:53,1,0.000312,0,0
2,id1217141,2016-06-30 23:59:47,1,0.000096,0,0
3,id2150126,2016-06-30 23:59:41,1,0.001319,1,0
4,id1598245,2016-06-30 23:59:33,1,0.000053,0,0


In [126]:
weekday = test_data[['pickup_datetime']].apply(find_weekday,axis=1)

In [127]:
tod = test_data[['pickup_datetime']].apply(find_tod,axis=1)

In [128]:
test_data.drop('pickup_datetime',axis=1, inplace=True)

In [129]:
weekday = pd.get_dummies(weekday)

In [130]:
tod = pd.get_dummies(tod)

In [131]:
weekday.drop(['0'],axis=1,inplace=True)

In [132]:
test_data = pd.concat([test_data,weekday,tod],axis=1)

In [133]:
test_data.drop('id',axis=1,inplace=True)

In [134]:
test_data.head()

,passenger_count,distance,2,Y,1,2,3,4,5,6,af,em,m,n
0,1,0.000306,0,0,0,0,0,1,0,0,0,0,0,1
1,1,0.000312,0,0,0,0,0,1,0,0,0,0,0,1
2,1,0.000096,0,0,0,0,0,1,0,0,0,0,0,1
3,1,0.001319,1,0,0,0,0,1,0,0,0,0,0,1
4,1,0.000053,0,0,0,0,0,1,0,0,0,0,0,1


In [135]:
X = test_data

In [136]:
from sklearn.preprocessing import StandardScaler

In [137]:
scaler = StandardScaler()

In [138]:
scaler.fit(X)
X = scaler.transform(X)

In [140]:
ans  = rfs.predict(X)

In [144]:
final_ans = pd.DataFrame(ans)

In [145]:
type(final_ans)

pandas.core.frame.DataFrame

In [146]:
final_ans.to_csv('pred.csv')

In [3]:
from math import exp

In [ ]:
exp()